In [12]:
import pandas as pd
import re
import glob
from datetime import datetime
import nltk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']= (17,5)
from collections import Counter
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import KMeans
from tqdm import tqdm
from nltk.corpus import PlaintextCorpusReader


In [14]:
#We here user %store to import, à la Python, our dataframe created in Sentiment Analysis Cluster.ipynb
#We then merge our dataframe with the lexical dataframe named lexical_2.csv, created with folder-lc-anc.py

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

%store -r df
df_lexical = pd.read_csv("assets/lexical_2.csv")
df = pd.merge(df, df_lexical, on='filename')
scaler = MinMaxScaler()
cols_to_norm = df.iloc[:, 16:].columns
df[cols_to_norm] = scaler.fit_transform(df[cols_to_norm])


'\nimport pandas as pd\nfrom sklearn.preprocessing import MinMaxScaler\n\n%store -r df\ndf_lexical = pd.read_csv("assets/lexical_2.csv")\ndf = pd.merge(df, df_lexical, on=\'filename\')\nscaler = MinMaxScaler()\ncols_to_norm = df.iloc[:, 16:].columns\ndf[cols_to_norm] = scaler.fit_transform(df[cols_to_norm])\n'

In [19]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_building_blocks as dbb
import plotly.express as px
import plotly.subplots as sp
import base64
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objs as go


%store -r df

class Graph(dbb.Block):

    def layout(self):
        return html.Div([
            dcc.Dropdown(
                id=self.register('dropdown'),
                options=self.data.options,
                value=self.data.value
            ),
            dcc.Graph(id=self.register('graph'))
        ], style={'width': '500'})

    def callbacks(self):
        @self.app.callback(
            self.output('graph', 'figure'),
            [self.input('dropdown', 'value')]
        )
        def update_charts(Debator):
            figure1_traces = []
            figure2_traces = []
            figure9_traces = []

            filtered_data = df[df["debater"] == Debator] #the graph/dataframe will be filterd by "Debater"
            scaler = MinMaxScaler()
            scatter = px.scatter(
                    filtered_data,
                    x="average_sentence_length",
                    y="debater",
                    size="number_of_sentences",
                    color="number_of_sentences",
                    color_continuous_scale=px.colors.sequential.Plotly3,
                    title="Something funny test 2",
            )
            scatter.update_layout(
                width =500, 
                xaxis_tickangle=30,
                title=dict(x=0.5),
                xaxis_tickfont=dict(size=9),
                yaxis_tickfont=dict(size=9),
                margin=dict(l=500, r=20, t=50, b=20),
                paper_bgcolor="LightSteelblue",
            )
            for trace in range(len(scatter["data"])):
                figure1_traces.append(scatter["data"][trace])
                
            #LEXICAL SOPHISTICATION
            filtered_data['average_sophistication'] = filtered_data[['ls2', 'cvs1', 'vs2']].mean(axis=1)
           
            fig = go.Figure()
            fig.add_trace(go.Bar(x=filtered_data.groupby("debater")["average_sophistication"].agg("median"),name='average'))
            fig.add_trace(go.Box(x=filtered_data["vs2"], name='vs2'))
            fig.add_trace(go.Box(x=filtered_data["cvs1"],name='cvs1'))
            fig.add_trace(go.Box(x=filtered_data["ls2"],name='ls2'))
            fig.update_traces(showlegend=False)
            for trace in range(len(fig["data"])):
                figure2_traces.append(fig["data"][trace])
            
            #LEXICAL DIVERSITY
            
            filtered_data['average_diversity'] = filtered_data[['ndw', 'cttr', 'cvv1', 'ttr', 'uber']].mean(axis=1)

            bar = go.Figure()
            bar.add_trace(go.Bar(x=filtered_data.groupby("debater")["average_diversity"].agg("median"),name='average'))
            bar.add_trace(go.Box(x=filtered_data["ndw"], name='ndw'))
            bar.add_trace(go.Box(x=filtered_data["cttr"],name='cttr'))
            bar.add_trace(go.Box(x=filtered_data["cvv1"],name='cvv1'))
            bar.add_trace(go.Box(x=filtered_data["ttr"],name='ttr'))
            bar.add_trace(go.Box(x=filtered_data["uber"],name='uber'))
            bar.update_traces(showlegend=False)
            for trace in range(len(bar["data"])):
                figure9_traces.append(bar["data"][trace])
            
            #The idea of trace is that we use subplots, in order to display graphs vertically, to fit our window nicely.
            this_figure = sp.make_subplots(rows=3, cols=1, subplot_titles=['Average Sentence Length', 'Lexical Sophistication', 'Lexical Diversity']) 
            for traces in figure1_traces:
                this_figure.append_trace(traces, row=1, col=1)
            for traces in figure2_traces:
                this_figure.append_trace(traces, row=2, col=1)
            for traces in figure9_traces:
                this_figure.append_trace(traces, row=3, col=1)

            this_figure.update_layout(
                width =800,
                height=1000,
            )
            
            return this_figure
        
        

app = JupyterDash(__name__, external_stylesheets=[
    "https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap-grid.min.css"
])

options = [
            {'label': debater, 'value':debater}
            for debater in df.debater.unique()
          ]

data = {
    'options': options,
    'value': 'Biden'
}

n_graphs = 2
graphs = [Graph(app, data) for _ in range(n_graphs)]

test_png = 'kul.png'
test_base64 = base64.b64encode(open(test_png, 'rb').read()).decode('ascii')
meme = 'meme.jpg'
meme_base64 = base64.b64encode(open(meme, 'rb').read()).decode('ascii')

for graph in graphs:
    graph.callbacks()

    
    
tab3 = html.Div(html.Iframe(src=('assets/great_speeches5.html'),
                            style=dict(position="relative", 
                                        left="0", 
                                        top="0",
                                        align='center',
                                        width="100%", 
                                        height="1000px"
                                        ),
                               ),

)

    
    
    
tab2 = dcc.Markdown('''
# Project's Goal

In a data driven world, we were asked to offer a new paradigm on what makes a great speech.\n
This question remains open to subjectivity, given how data can be biased, yet we will try to offer a new paradigm to answer this question. 
\n Rather than giving a (insignificant) binary outcome through a classical logistic regression, we will open the blackbox and offer the reader additional tools to decide what could make a great speech:

- A topic modeling approach
- A lexical approach
- A clustering approach

## Features

- To avoid survivorship bias, we merge exceptional political speeches with recent US presidential debates.
- We preprocess our text, using a stopword list, and a POS TAGGING methodology when needed.
- We assess the hypothesis and compute the cluster / topic modeling / lexical measures.
- We display this information in a simple manner, allowing comparison. 

As Thomas Carlyle once wrote:

> A man lives by believing something: not by debating and arguing about many things.



## Contributors

Dimitri Soccol \n
Amar Music \n
Adam Russell \n
Huu Duc Luu \n
Thomas Umberto Grava
''')   

tab1 = html.Div(
            children=[
                html.Div(
                    [html.Div(graph.layout, className='container')
                    for graph in graphs],
                    className='six.columns',
                    style = {'display' : 'flex'}
                ),
            ],
    )

 
    
app.layout = html.Div(
        children=[
                html.Div(
                        children=[
                            html.Img(src='data:image/png;base64,{}'.format(test_base64),width=100),
                            html.H1(
                                children="Political speeches analysis",style={'textAlign': 'center'}, className="header-title" 
                            ), #Header title
                        ],
                        className="header",style={'textAlign': 'center'},
                    ),
        html.Div([
            dcc.Tabs(id="tabs-example", value='tab-2-example', children=[
                dcc.Tab(id="tab-2", label='Introduction', value='tab-2-example'),
                dcc.Tab(id="tab-1", label='Analysis', value='tab-1-example'),
                dcc.Tab(id="tab-3", label='Topic Modeling', value='tab-3-example'),
            ]),
            html.Div(id='tabs-content-example',
                     children = tab1)
        ])
    ])

@app.callback(dash.dependencies.Output('tabs-content-example', 'children'),
             [dash.dependencies.Input('tabs-example', 'value')])
def render_content(tab):
    if tab == 'tab-2-example':
        return tab2
    elif tab == 'tab-1-example':
        return tab1
    elif tab == 'tab-3-example':
        return tab3

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

#Host 0.0.0.0 because we run our Dash app on this server.
if __name__ == '__main__':
    app.run_server(host='0.0.0.0',debug=True, port=8003)

Dash app running on http://0.0.0.0:8003/


/root/venv1/lib/python3.8/site-packages/dash/resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources

/tmp/ipykernel_3559867/2891273209.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3559867/2891273209.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

/tmp/ipykernel_3559867/2891273209.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3559867/2891273209.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3559867/2891273209.py:77: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ip

/tmp/ipykernel_3559867/2891273209.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3559867/2891273209.py:77: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3559867/2891273209.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ip